In [4]:
#
# hw4pr1iris:  iris clasification via nearest neighbors
#

In [5]:
#
# We don't need any data at all to create a predictive model!
#
import random

def predictive_model( Features ):
    """ input: a list of four features 
                [ sepallen, sepalwid, petallen, petalwid ]
        output: the predicted species of iris, from
                  setosa (0), versicolor (1), virginica (2)
    """
    [ sepallen, sepalwid, petallen, petalwid ] = Features # unpacking!
    if petalwid < 1.0:
        return 'setosa (0)'
    else:
        return random.choice( ['versicolor (1)', 'virginica (2)'] )
    
#
# Try it!
# 
# Features = eval(input("Enter new Features: "))
#
Features = [ 4.6, 3.6, 3.0, 0.2 ] 
result = predictive_model( Features )
print(f"I predict {result} from Features {Features}")

I predict setosa (0) from Features [4.6, 3.6, 3.0, 0.2]


In [6]:
#
# (now, to explore how we _can_ use data to do better... :-) 
#

In [7]:
# libraries!
import numpy as np      # numpy is Python's "array" library
import pandas as pd     # Pandas is Python's "data" library ("dataframe" == spreadsheet)

In [8]:
# let's read in our flower data...
# 
# for read_csv, use header=0 when row 0 is a header row
# 
filename = 'iris.csv'
df = pd.read_csv(filename, header=0)   # encoding="latin1" et al.
print(f"{filename} : file read into a pandas dataframe.")

iris.csv : file read into a pandas dataframe.


In [9]:
#
# a dataframe is a "spreadsheet in Python"   (seems to have an extra column!)
#
pd.set_option('display.max_rows', 10)  # None for no limit; default: 10
pd.set_option('display.min_rows', 10)  # None for no limit; default: 10
# let's view it!
df

,sepallen,sepalwid,petallen,petalwid,irisname,adapted from https://en.wikipedia.org/wiki/Iris_flower_data_set
0,4.6,3.6,1.0,2.000000e-01,setosa,NaN
1,4.3,3.0,1.1,1.000000e-01,setosa,NaN
2,5.0,3.2,1.2,2.000000e-01,setosa,NaN
3,5.8,4.0,1.2,2.000000e-01,setosa,NaN
4,4.4,3.0,1.3,2.000000e-01,setosa,NaN
...,...,...,...,...,...,...
138,7.7,3.8,6.7,2.200000e+00,virginica,NaN
139,7.7,2.8,6.7,2.000000e+00,virginica,NaN
140,7.7,2.6,6.9,2.300000e+00,virginica,NaN
141,4.2,42.0,4242.0,4.200000e+42,alieniris,NaN


In [10]:
#
# let's look at our pandas dataframe   (Aargh: that extra column!)
#
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143 entries, 0 to 142
Data columns (total 6 columns):
 #   Column                                                           Non-Null Count  Dtype  
---  ------                                                           --------------  -----  
 0   sepallen                                                         143 non-null    float64
 1   sepalwid                                                         143 non-null    float64
 2   petallen                                                         143 non-null    float64
 3   petalwid                                                         143 non-null    float64
 4   irisname                                                         142 non-null    object 
 5   adapted from https://en.wikipedia.org/wiki/Iris_flower_data_set  0 non-null      float64
dtypes: float64(5), object(1)
memory usage: 6.8+ KB


In [11]:
#
# let's drop that last column (dropping is usually by _name_):
#
#   if you want a list of the column names use df.columns
col5name = df.columns[5]  # get column name at index 5
df_clean = df.drop(columns=[col5name])  # drop by name is typical
df_clean.info()                         # should be happier!

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143 entries, 0 to 142
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   sepallen  143 non-null    float64
 1   sepalwid  143 non-null    float64
 2   petallen  143 non-null    float64
 3   petalwid  143 non-null    float64
 4   irisname  142 non-null    object 
dtypes: float64(4), object(1)
memory usage: 5.7+ KB


In [12]:
#
# let's keep our column names in variables, for reference
#
COLUMNS = df_clean.columns            # "list" of columns
print(f"COLUMNS is {COLUMNS}\n")  
  # It's a "pandas" list, called an Index
  # use it just as a Python list of strings:
print(f"COLUMNS[0] is {COLUMNS[0]}\n")

# let's create a dictionary to look up any column index by name
COL_INDEX = {}
for i, name in enumerate(COLUMNS):
    COL_INDEX[name] = i  # using the name (as key), look up the value (i)
print(f"COL_INDEX is {COL_INDEX}")

COLUMNS is Index(['sepallen', 'sepalwid', 'petallen', 'petalwid', 'irisname'], dtype='object')

COLUMNS[0] is sepallen

COL_INDEX is {'sepallen': 0, 'sepalwid': 1, 'petallen': 2, 'petalwid': 3, 'irisname': 4}


In [13]:
#
# let's look at our cleaned-up dataframe...
#
df_clean.info()   
#
# notice that the non-null is _different_ for irisname!
df_clean   # show a table! (the problem rows are the last two...)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143 entries, 0 to 142
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   sepallen  143 non-null    float64
 1   sepalwid  143 non-null    float64
 2   petallen  143 non-null    float64
 3   petalwid  143 non-null    float64
 4   irisname  142 non-null    object 
dtypes: float64(4), object(1)
memory usage: 5.7+ KB


,sepallen,sepalwid,petallen,petalwid,irisname
0,4.6,3.6,1.0,2.000000e-01,setosa
1,4.3,3.0,1.1,1.000000e-01,setosa
2,5.0,3.2,1.2,2.000000e-01,setosa
3,5.8,4.0,1.2,2.000000e-01,setosa
4,4.4,3.0,1.3,2.000000e-01,setosa
...,...,...,...,...,...
138,7.7,3.8,6.7,2.200000e+00,virginica
139,7.7,2.8,6.7,2.000000e+00,virginica
140,7.7,2.6,6.9,2.300000e+00,virginica
141,4.2,42.0,4242.0,4.200000e+42,alieniris


In [14]:
#
# typically, after dropping columns we don't want, 
#   we drop rows with missing data (other approaches are possible, too)
#
df_full = df_clean.dropna()   # this removes all rows with nan items
df_full.info()                # it's "full" because it has no nan items
df_full
#
# notice that _all_ of the rows now have 142 non-null items
#    also, the last row isn't real data... we'll handle it next

<class 'pandas.core.frame.DataFrame'>
Int64Index: 142 entries, 0 to 141
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   sepallen  142 non-null    float64
 1   sepalwid  142 non-null    float64
 2   petallen  142 non-null    float64
 3   petalwid  142 non-null    float64
 4   irisname  142 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.7+ KB


,sepallen,sepalwid,petallen,petalwid,irisname
0,4.6,3.6,1.0,2.000000e-01,setosa
1,4.3,3.0,1.1,1.000000e-01,setosa
2,5.0,3.2,1.2,2.000000e-01,setosa
3,5.8,4.0,1.2,2.000000e-01,setosa
4,4.4,3.0,1.3,2.000000e-01,setosa
...,...,...,...,...,...
137,7.6,3.0,6.6,2.100000e+00,virginica
138,7.7,3.8,6.7,2.200000e+00,virginica
139,7.7,2.8,6.7,2.000000e+00,virginica
140,7.7,2.6,6.9,2.300000e+00,virginica


In [15]:
# 
# get rid of last row!
#
df_final = df_full.iloc[0:-1]   # not the syntax I would choose
# careful:  don't run this again!
print(df_final.shape)
df_final

(141, 5)


,sepallen,sepalwid,petallen,petalwid,irisname
0,4.6,3.6,1.0,0.2,setosa
1,4.3,3.0,1.1,0.1,setosa
2,5.0,3.2,1.2,0.2,setosa
3,5.8,4.0,1.2,0.2,setosa
4,4.4,3.0,1.3,0.2,setosa
...,...,...,...,...,...
136,7.9,3.8,6.4,2.0,virginica
137,7.6,3.0,6.6,2.1,virginica
138,7.7,3.8,6.7,2.2,virginica
139,7.7,2.8,6.7,2.0,virginica


In [16]:
# all of scikit-learn's ML routines need numbers, not strings
#   ... even for categories/classifications (like species!)
#   so, we will convert the flower-species to numbers:

SPECIES = ['setosa','versicolor','virginica']   # int to str
SPECIES_INDEX = {'setosa':0,'versicolor':1,'virginica':2}  # str to int

def convert_species(speciesname):
    """ return the species index (a unique integer/category) """
    #print(f"converting {speciesname}...")
    return SPECIES_INDEX[speciesname]

# Let's try it out...
for name in SPECIES:
    print(f"{name} maps to {convert_species(name)}")

setosa maps to 0
versicolor maps to 1
virginica maps to 2


In [17]:
#
# we can "apply" to a whole column
#   it may give a warning, but this is ok...
#

df_final['irisname'] = df_final['irisname'].apply(convert_species)

# Don't run this twice!   Why?!  What's "KeyError: 0"?
#   (for sure, you can always go back and re-establish definitions)

# don't worry about the (possible)  "SettingWithCopyWarning" here...

<ipython-input-17-93da7aa072d5>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['irisname'] = df_final['irisname'].apply(convert_species)


In [18]:
#
# let's see it!  (this is safe to run many times...)
#
df_final         # print(df_final.tostring())  # for _all_ rows...

,sepallen,sepalwid,petallen,petalwid,irisname
0,4.6,3.6,1.0,0.2,0
1,4.3,3.0,1.1,0.1,0
2,5.0,3.2,1.2,0.2,0
3,5.8,4.0,1.2,0.2,0
4,4.4,3.0,1.3,0.2,0
...,...,...,...,...,...
136,7.9,3.8,6.4,2.0,2
137,7.6,3.0,6.6,2.1,2
138,7.7,3.8,6.7,2.2,2
139,7.7,2.8,6.7,2.0,2


In [19]:
#
# let's convert our dataframe to a numpy array, named A
#    Our ML library, scikit-learn operates entirely on numpy arrays.
#
A = df_final.values    # .values gets the numpy array
print(A)

[[4.6 3.6 1.  0.2 0. ]
 [4.3 3.  1.1 0.1 0. ]
 [5.  3.2 1.2 0.2 0. ]
 [5.8 4.  1.2 0.2 0. ]
 [4.4 3.  1.3 0.2 0. ]
 [4.4 3.2 1.3 0.2 0. ]
 [4.5 2.3 1.3 0.3 0. ]
 [4.7 3.2 1.3 0.2 0. ]
 [5.  3.5 1.3 0.3 0. ]
 [5.4 3.9 1.3 0.4 0. ]
 [5.5 3.5 1.3 0.2 0. ]
 [4.4 2.9 1.4 0.2 0. ]
 [4.6 3.4 1.4 0.3 0. ]
 [4.6 3.2 1.4 0.2 0. ]
 [4.8 3.  1.4 0.1 0. ]
 [4.8 3.  1.4 0.3 0. ]
 [4.9 3.  1.4 0.2 0. ]
 [5.  3.6 1.4 0.2 0. ]
 [5.  3.3 1.4 0.2 0. ]
 [5.1 3.5 1.4 0.2 0. ]
 [5.1 3.5 1.4 0.3 0. ]
 [5.2 3.4 1.4 0.2 0. ]
 [5.5 4.2 1.4 0.2 0. ]
 [4.6 3.1 1.5 0.2 0. ]
 [4.9 3.1 1.5 0.1 0. ]
 [4.9 3.1 1.5 0.1 0. ]
 [4.9 3.1 1.5 0.1 0. ]
 [5.  3.4 1.5 0.2 0. ]
 [5.1 3.8 1.5 0.3 0. ]
 [5.1 3.7 1.5 0.4 0. ]
 [5.1 3.4 1.5 0.2 0. ]
 [5.2 3.5 1.5 0.2 0. ]
 [5.3 3.7 1.5 0.2 0. ]
 [5.4 3.7 1.5 0.2 0. ]
 [5.7 4.4 1.5 0.4 0. ]
 [4.7 3.2 1.6 0.2 0. ]
 [4.8 3.4 1.6 0.2 0. ]
 [5.  3.  1.6 0.2 0. ]
 [5.  3.4 1.6 0.4 0. ]
 [5.  3.5 1.6 0.6 0. ]
 [5.1 3.8 1.6 0.2 0. ]
 [5.1 3.3 1.7 0.5 0. ]
 [5.4 3.9 1.7 0.4 0. ]
 [5.4 3.4 1

In [20]:
#
# let's make sure it's all floating-point, so we can multiply and divide
#
A = A.astype('float64')  # so many:  www.tutorialspoint.com/numpy/numpy_data_types.htm
print(A)

[[4.6 3.6 1.  0.2 0. ]
 [4.3 3.  1.1 0.1 0. ]
 [5.  3.2 1.2 0.2 0. ]
 [5.8 4.  1.2 0.2 0. ]
 [4.4 3.  1.3 0.2 0. ]
 [4.4 3.2 1.3 0.2 0. ]
 [4.5 2.3 1.3 0.3 0. ]
 [4.7 3.2 1.3 0.2 0. ]
 [5.  3.5 1.3 0.3 0. ]
 [5.4 3.9 1.3 0.4 0. ]
 [5.5 3.5 1.3 0.2 0. ]
 [4.4 2.9 1.4 0.2 0. ]
 [4.6 3.4 1.4 0.3 0. ]
 [4.6 3.2 1.4 0.2 0. ]
 [4.8 3.  1.4 0.1 0. ]
 [4.8 3.  1.4 0.3 0. ]
 [4.9 3.  1.4 0.2 0. ]
 [5.  3.6 1.4 0.2 0. ]
 [5.  3.3 1.4 0.2 0. ]
 [5.1 3.5 1.4 0.2 0. ]
 [5.1 3.5 1.4 0.3 0. ]
 [5.2 3.4 1.4 0.2 0. ]
 [5.5 4.2 1.4 0.2 0. ]
 [4.6 3.1 1.5 0.2 0. ]
 [4.9 3.1 1.5 0.1 0. ]
 [4.9 3.1 1.5 0.1 0. ]
 [4.9 3.1 1.5 0.1 0. ]
 [5.  3.4 1.5 0.2 0. ]
 [5.1 3.8 1.5 0.3 0. ]
 [5.1 3.7 1.5 0.4 0. ]
 [5.1 3.4 1.5 0.2 0. ]
 [5.2 3.5 1.5 0.2 0. ]
 [5.3 3.7 1.5 0.2 0. ]
 [5.4 3.7 1.5 0.2 0. ]
 [5.7 4.4 1.5 0.4 0. ]
 [4.7 3.2 1.6 0.2 0. ]
 [4.8 3.4 1.6 0.2 0. ]
 [5.  3.  1.6 0.2 0. ]
 [5.  3.4 1.6 0.4 0. ]
 [5.  3.5 1.6 0.6 0. ]
 [5.1 3.8 1.6 0.2 0. ]
 [5.1 3.3 1.7 0.5 0. ]
 [5.4 3.9 1.7 0.4 0. ]
 [5.4 3.4 1

In [21]:
#
# nice to have NUM_ROWS and NUM_COLS around
#
NUM_ROWS, NUM_COLS = A.shape
print(f"\nThe dataset has {NUM_ROWS} rows and {NUM_COLS} cols")


The dataset has 141 rows and 5 cols


In [22]:
# let's use all of our variables, to reinforce names...

# choose a row index, n:
n = 132
print(f"flower #{n} is {A[n]}")

for i in range(len(COLUMNS)):
    colname = COLUMNS[i]
    if colname != 'irisname':
        print(f"  Its {colname} is {A[n][i]}")
    else:
        species_num = int(A[n][i])
        species = SPECIES[species_num]
        print(f"  Its {colname} is {species} ({species_num})")

flower #132 is [7.2 3.6 6.1 2.5 2. ]
  Its sepallen is 7.2
  Its sepalwid is 3.6
  Its petallen is 6.1
  Its petalwid is 2.5
  Its irisname is virginica (2)


In [23]:
#
# We don't have to use scikit-learn to implement n.n.!
#

#
# data-driven predictive model (1-nearest-neighbor)
#
dist = np.linalg.norm  # built in to numpy
NUM_ROWS, NUM_COLS = A.shape  # data size

def predictive_model( Features ):
    """ input: a list of four features 
                [ sepallen, sepalwid, petallen, petalwid ]
        output: the predicted species of iris, from
                  setosa (0), versicolor (1), virginica (2)
    """
    our_features = np.asarray(Features)   # make a numpy array
    
    closest_flower   = A[0]
    closest_features = A[0,0:4]         
    closest_distance = dist(our_features-closest_features)
    
    for i in range(NUM_ROWS):
        current_flower   = A[i]
        current_features = A[i,0:4] 
        current_distance = dist(our_features-current_features)
        
        if current_distance < closest_distance:
            closest_distance = current_distance  # remember closest!
            closest_flower = current_flower
    
    # done comparing with every flower in the dataset
    predicted_species = int(round(closest_flower[4]))
    name = SPECIES[predicted_species]
    return f"{name} ({predicted_species})"
    
#
# Try it!
# 
# Features = eval(input("Enter new Features: "))
#
Features = [ 4.6, 3.6, 3.0, 1.2 ] 
result = predictive_model( Features )
print(f"I predict {result} from Features {Features}")

I predict versicolor (1) from Features [4.6, 3.6, 3.0, 1.2]


In [24]:
#
# but, we don't have to write our own ... because
#
#     we want knn for any k!
#     we want an already-debugged algorithm!
#     we want to ask iris q'ns instead of implementation ones... (?)
#

In [25]:
print("+++ Start of data definitions +++\n")

X_all = A[:,0:4]  # X (features) ... is all rows, columns 0, 1, 2, 3
y_all = A[:,4]    # y (labels) ... is all rows, column 4 only

print(f"X_all (just features) is \n {X_all}")
print(f"y_all (just labels)   is \n {y_all}")

+++ Start of data definitions +++

X_all (just features) is 
 [[4.6 3.6 1.  0.2]
 [4.3 3.  1.1 0.1]
 [5.  3.2 1.2 0.2]
 [5.8 4.  1.2 0.2]
 [4.4 3.  1.3 0.2]
 [4.4 3.2 1.3 0.2]
 [4.5 2.3 1.3 0.3]
 [4.7 3.2 1.3 0.2]
 [5.  3.5 1.3 0.3]
 [5.4 3.9 1.3 0.4]
 [5.5 3.5 1.3 0.2]
 [4.4 2.9 1.4 0.2]
 [4.6 3.4 1.4 0.3]
 [4.6 3.2 1.4 0.2]
 [4.8 3.  1.4 0.1]
 [4.8 3.  1.4 0.3]
 [4.9 3.  1.4 0.2]
 [5.  3.6 1.4 0.2]
 [5.  3.3 1.4 0.2]
 [5.1 3.5 1.4 0.2]
 [5.1 3.5 1.4 0.3]
 [5.2 3.4 1.4 0.2]
 [5.5 4.2 1.4 0.2]
 [4.6 3.1 1.5 0.2]
 [4.9 3.1 1.5 0.1]
 [4.9 3.1 1.5 0.1]
 [4.9 3.1 1.5 0.1]
 [5.  3.4 1.5 0.2]
 [5.1 3.8 1.5 0.3]
 [5.1 3.7 1.5 0.4]
 [5.1 3.4 1.5 0.2]
 [5.2 3.5 1.5 0.2]
 [5.3 3.7 1.5 0.2]
 [5.4 3.7 1.5 0.2]
 [5.7 4.4 1.5 0.4]
 [4.7 3.2 1.6 0.2]
 [4.8 3.4 1.6 0.2]
 [5.  3.  1.6 0.2]
 [5.  3.4 1.6 0.4]
 [5.  3.5 1.6 0.6]
 [5.1 3.8 1.6 0.2]
 [5.1 3.3 1.7 0.5]
 [5.4 3.9 1.7 0.4]
 [5.4 3.4 1.7 0.2]
 [5.7 3.8 1.7 0.3]
 [4.8 3.4 1.9 0.2]
 [5.1 3.8 1.9 0.4]
 [7.  3.2 4.7 1.4]
 [4.9 2.4 3.3 1. ]
 [5.  2

In [26]:
#
# we can re-weight different features here...
#

COL_WEIGHT = {              # could be called Feature weight...
    'sepallen':1.0,
    'sepalwid':1.0,
    'petallen':1.0,
    'petalwid':1.0,
}

for colname in COL_WEIGHT:
    i = COL_INDEX[colname]    # get the column index, i, of the colname
    weight = COL_WEIGHT[colname]  # from the dictionary above
    print("Weighting", colname, "by", weight)   
    # weighting == "multiplying"
    X_all[:,i] *= weight   # multiply by the weight to give this column ("feature")

Weighting sepallen by 1.0
Weighting sepalwid by 1.0
Weighting petallen by 1.0
Weighting petalwid by 1.0


In [27]:
#
# we scramble the data, to give a different TRAIN/TEST split each time...
# 
indices = np.random.permutation(len(y_all))  # indices is a permutation-list

# we scramble both X and y, necessarily with the same permutation
X_labeled = X_all[indices]              # we apply the _same_ permutation to each!
y_labeled = y_all[indices]              # again...
print(X_labeled)
print(y_labeled)

[[6.2 2.2 4.5 1.5]
 [5.5 2.6 4.4 1.2]
 [5.5 2.5 4.  1.3]
 [6.2 2.8 4.8 1.8]
 [5.4 3.9 1.3 0.4]
 [7.2 3.6 6.1 2.5]
 [6.4 2.8 5.6 2.1]
 [5.5 3.5 1.3 0.2]
 [5.7 2.6 3.5 1. ]
 [5.5 2.3 4.  1.3]
 [6.3 2.5 5.  1.9]
 [5.6 2.5 3.9 1.1]
 [5.3 3.7 1.5 0.2]
 [5.1 3.5 1.4 0.3]
 [5.8 2.6 4.  1.2]
 [7.  3.2 4.7 1.4]
 [6.7 3.1 4.7 1.5]
 [5.  3.4 1.5 0.2]
 [4.7 3.2 1.6 0.2]
 [7.6 3.  6.6 2.1]
 [5.1 3.5 1.4 0.2]
 [5.1 3.8 1.6 0.2]
 [7.7 3.  6.1 2.3]
 [6.5 3.  5.8 2.2]
 [5.  3.5 1.6 0.6]
 [6.9 3.1 5.1 2.3]
 [4.9 2.5 4.5 1.7]
 [6.3 2.9 5.6 1.8]
 [7.3 2.9 6.3 1.8]
 [4.7 3.2 1.3 0.2]
 [6.  3.  4.8 1.8]
 [6.7 3.1 5.6 2.4]
 [6.5 3.2 5.1 2. ]
 [6.8 2.8 4.8 1.4]
 [6.4 3.2 4.5 1.5]
 [5.5 2.4 3.8 1.1]
 [6.  3.4 4.5 1.6]
 [5.4 3.4 1.7 0.2]
 [6.3 3.3 4.7 1.6]
 [6.7 3.  5.  1.7]
 [4.3 3.  1.1 0.1]
 [6.1 2.6 5.6 1.4]
 [5.6 2.9 3.6 1.3]
 [6.4 3.1 5.5 1.8]
 [6.3 3.4 5.6 2.4]
 [7.7 2.8 6.7 2. ]
 [7.4 2.8 6.1 1.9]
 [5.8 2.7 3.9 1.2]
 [5.2 3.5 1.5 0.2]
 [5.8 4.  1.2 0.2]
 [5.  2.  3.5 1. ]
 [6.1 2.8 4.  1.3]
 [6.9 3.1 5.

In [28]:
#
# We next separate into test data and training data ... 
#    + We will train on the training data...
#    + We will _not_ look at the testing data to build the model
#
# Then, afterward, we will test on the testing data -- and see how well we do!
#

#
# a common convention:  train on 80%, test on 20%    Let's define the TEST_PERCENT
#
NUM_ROWS = X_labeled.shape[0]     # the number of labeled rows
TEST_PERCENT = 0.20
TEST_SIZE = int(TEST_PERCENT*NUM_ROWS)   # no harm in rounding down

X_test = X_labeled[:TEST_SIZE]    # first section are for testing
y_test = y_labeled[:TEST_SIZE]

X_train = X_labeled[TEST_SIZE:]   # all the rest are for training
y_train = y_labeled[TEST_SIZE:]

print(f"training with {len(y_train)} rows;  testing with {len(y_test)} rows" )

training with 113 rows;  testing with 28 rows


In [29]:
#
# +++ This is the "Model-building and Model-training Cell"
#       
# Create a kNN model and train it! 
#
from sklearn.neighbors import KNeighborsClassifier

k = 84   # we don't know what k to use, so we guess!  (this will _not_ be a good value)
knn_model = KNeighborsClassifier(n_neighbors=k)       # here, k is the "k" in kNN

# we train the model (it's one line!)
knn_model.fit(X_train, y_train)                              # yay!  trained!
print("Created and trained a knn classifier with k =", k)  

Created and trained a knn classifier with k = 84


In [30]:
#
# +++ This is the "Model-testing Cell"
#
# Now, let's see how well we did on our "held-out data" (the testing data)
#

# We run our test set!
predicted_labels = knn_model.predict(X_test)
actual_labels = y_test

# Let's print them so we can compare...
print("Predicted labels:", predicted_labels)
print("Actual  labels  :", actual_labels)

# And, some overall results
num_correct = sum(predicted_labels == actual_labels)
total = len(actual_labels)
print(f"\nResults on test set:  {num_correct} correct out of {total} total.")

Predicted labels: [2. 1. 1. 2. 0. 2. 2. 0. 1. 1. 2. 1. 0. 0. 1. 2. 2. 0. 0. 2. 0. 0. 2. 2.
 0. 2. 1. 2.]
Actual  labels  : [1. 1. 1. 2. 0. 2. 2. 0. 1. 1. 2. 1. 0. 0. 1. 1. 1. 0. 0. 2. 0. 0. 2. 2.
 0. 2. 2. 2.]

Results on test set:  24 correct out of 28 total.


In [31]:
#
# Let's print these more helpfully, in a vertical table
#

def compare_labels(predicted_labels, actual_labels):
    """ a more neatly formatted comparison """
    NUM_LABELS = len(predicted_labels)
    num_correct = 0
    
    for i in range(NUM_LABELS):
        p = int(round(predicted_labels[i]))         # round protects from fp error 
        a = int(round(actual_labels[i]))
        result = "incorrect"
        if p == a:  # if they match,
            result = ""       # no longer incorrect
            num_correct += 1  # and we count a match!

        print(f"row {i:>3d} : {SPECIES[p]:>12s} {SPECIES[a]:<12s}   {result}")   

    print()
    print("Correct:", num_correct, "out of", NUM_LABELS)
    return num_correct

In [32]:
#
# let's try it out!
#

compare_labels(predicted_labels,actual_labels)

row   0 :    virginica versicolor     incorrect
row   1 :   versicolor versicolor     
row   2 :   versicolor versicolor     
row   3 :    virginica virginica      
row   4 :       setosa setosa         
row   5 :    virginica virginica      
row   6 :    virginica virginica      
row   7 :       setosa setosa         
row   8 :   versicolor versicolor     
row   9 :   versicolor versicolor     
row  10 :    virginica virginica      
row  11 :   versicolor versicolor     
row  12 :       setosa setosa         
row  13 :       setosa setosa         
row  14 :   versicolor versicolor     
row  15 :    virginica versicolor     incorrect
row  16 :    virginica versicolor     incorrect
row  17 :       setosa setosa         
row  18 :       setosa setosa         
row  19 :    virginica virginica      
row  20 :       setosa setosa         
row  21 :       setosa setosa         
row  22 :    virginica virginica      
row  23 :    virginica virginica      
row  24 :       setosa setosa        

24

In [33]:
#
# Ok!  We have our knn model, we could just use it...
#

#
# data-driven predictive model (k-nearest-neighbor), using scikit-learn
#

def predictive_model( Features ):
    """ input: a list of four features 
                [ sepallen, sepalwid, petallen, petalwid ]
        output: the predicted species of iris, from
                  setosa (0), versicolor (1), virginica (2)
    """
    our_features = np.asarray([Features])                 # extra brackets needed
    predicted_species = knn_model.predict(our_features)
    
    predicted_species = int(round(predicted_species[0]))  # unpack one element
    name = SPECIES[predicted_species]
    return f"{name} ({predicted_species})"
    
#
# Try it!
# 
# Features = eval(input("Enter new Features: "))
#
Features = [6.7,3.3,5.7,2.1]  # [5.8,2.7,4.1,1.0] [4.6,3.6,3.0,2.2] [6.7,3.3,5.7,2.1]
result = predictive_model( Features )
print(f"I predict {result} from Features {Features}")

I predict virginica (2) from Features [6.7, 3.3, 5.7, 2.1]


In [34]:
#
# Except, we didn't really explore whether this was the BEST model we could build!
#
#
# We used k = 84  (a neighborhood size of 84 flowers)
# In a dataset of only 140ish flowers, with three species, this seems like a bad idea!
#
# Perhaps we should try ALL the neighborhood sizes in their own TRAIN/TEST split
# and see which neighborhood size works the best, for irises, at least...
#

In [44]:
#
# to do this, we use "cross validation"
#

from sklearn.model_selection import cross_val_score

#
# cross-validation splits the training set into two pieces:
#   + model-building and model-validation. We'll use "build" and "validate"
#
best_k = 0
best_accuracy = 0
for k in range(1,85):
    knn_cv_model = KNeighborsClassifier(n_neighbors=k)   # build knn_model for every k!
    cv_scores = cross_val_score( knn_cv_model, X_train, y_train, cv=5 )  # 5 means 80/20 split
    # print(cv_scores)  # just to see the five scores... 
    average_cv_accuracy = cv_scores.mean()  # mean() is numpy's built-in average function 
    if average_cv_accuracy > best_accuracy:
        best_k = k
        best_accuracy = average_cv_accuracy
    
    print(f"k: {k:2d}  cv accuracy: {average_cv_accuracy:7.4f}")

    
    
# assign best value of k to best_k
# you'll need to use the loop above to find and remember the real best_k

print(f"best_k = {best_k}   yields the highest average cv accuracy.")  # print the best one

k:  1  cv accuracy:  0.9466
k:  2  cv accuracy:  0.9379
k:  3  cv accuracy:  0.9557
k:  4  cv accuracy:  0.9557
k:  5  cv accuracy:  0.9648
k:  6  cv accuracy:  0.9561
k:  7  cv accuracy:  0.9561
k:  8  cv accuracy:  0.9379
k:  9  cv accuracy:  0.9470
k: 10  cv accuracy:  0.9470
k: 11  cv accuracy:  0.9648
k: 12  cv accuracy:  0.9379
k: 13  cv accuracy:  0.9466
k: 14  cv accuracy:  0.9379
k: 15  cv accuracy:  0.9379
k: 16  cv accuracy:  0.9379
k: 17  cv accuracy:  0.9466
k: 18  cv accuracy:  0.9466
k: 19  cv accuracy:  0.9466
k: 20  cv accuracy:  0.9289
k: 21  cv accuracy:  0.9289
k: 22  cv accuracy:  0.9289
k: 23  cv accuracy:  0.9379
k: 24  cv accuracy:  0.9379
k: 25  cv accuracy:  0.9379
k: 26  cv accuracy:  0.9285
k: 27  cv accuracy:  0.9379
k: 28  cv accuracy:  0.9285
k: 29  cv accuracy:  0.9375
k: 30  cv accuracy:  0.9289
k: 31  cv accuracy:  0.9375
k: 32  cv accuracy:  0.9375
k: 33  cv accuracy:  0.9289
k: 34  cv accuracy:  0.9198
k: 35  cv accuracy:  0.9198
k: 36  cv accuracy: 

In [45]:
#
# Now, we re-create and re-run the  "Model-building and -training Cell"
#
# Now, using best_k instead of the original, randomly-guessed value    How does it do?!
#
from sklearn.neighbors import KNeighborsClassifier
knn_model_tuned = KNeighborsClassifier(n_neighbors=best_k)   # here, we use the best_k

# we train the model (one line!)
knn_model_tuned.fit(X_train, y_train)                              # yay!  trained!
print(f"Created + trained a knn classifier, now tuned with a (best) k of {best_k}")  

Created + trained a knn classifier, now tuned with a (best) k of 5


In [46]:
#
# Re-create and re-run the  "Model-testing Cell"     How does it do with best_k?!
#
predicted_labels = knn_model_tuned.predict(X_test)
actual_labels = y_test

# Let's print them so we can compare...
print("Predicted labels:", predicted_labels)
print("Actual labels:", actual_labels)
print()
# and, we'll print our nicer table...
compare_labels(predicted_labels,actual_labels)

Predicted labels: [1. 1. 1. 2. 0. 2. 2. 0. 1. 1. 2. 1. 0. 0. 1. 1. 1. 0. 0. 2. 0. 0. 2. 2.
 0. 2. 1. 2.]
Actual labels: [1. 1. 1. 2. 0. 2. 2. 0. 1. 1. 2. 1. 0. 0. 1. 1. 1. 0. 0. 2. 0. 0. 2. 2.
 0. 2. 2. 2.]

row   0 :   versicolor versicolor     
row   1 :   versicolor versicolor     
row   2 :   versicolor versicolor     
row   3 :    virginica virginica      
row   4 :       setosa setosa         
row   5 :    virginica virginica      
row   6 :    virginica virginica      
row   7 :       setosa setosa         
row   8 :   versicolor versicolor     
row   9 :   versicolor versicolor     
row  10 :    virginica virginica      
row  11 :   versicolor versicolor     
row  12 :       setosa setosa         
row  13 :       setosa setosa         
row  14 :   versicolor versicolor     
row  15 :   versicolor versicolor     
row  16 :   versicolor versicolor     
row  17 :       setosa setosa         
row  18 :       setosa setosa         
row  19 :    virginica virginica      
row  20 :   

27

In [47]:
#
# Ok!  We have tuned knn to use the "best" value of k...
#
# And, we should really use ALL available data to train our final predictive model:
#

knn_model_final = KNeighborsClassifier(n_neighbors=best_k)   # here, we use the best_k
knn_model_final.fit(X_all, y_all)                              # yay!  trained!
print(f"Created + trained a 'final' knn classifier, with a (best) k of {best_k}") 

Created + trained a 'final' knn classifier, with a (best) k of 5


In [48]:
#
# final predictive model (k-nearest-neighbor), with tuned k + ALL data incorporated
#

def predictive_model( Features ):
    """ input: a list of four features 
                [ sepallen, sepalwid, petallen, petalwid ]
        output: the predicted species of iris, from
                  setosa (0), versicolor (1), virginica (2)
    """
    our_features = np.asarray([Features])                 # extra brackets needed
    predicted_species = knn_model_final.predict(our_features)
    
    predicted_species = int(round(predicted_species[0]))  # unpack one element
    name = SPECIES[predicted_species]
    return f"{name} ({predicted_species})"
    
#
# Try it!
# 
# Features = eval(input("Enter new Features: "))
#
Features = [6.7,3.3,5.7,2.1]  # [5.8,2.7,4.1,1.0] [4.6,3.6,3.0,2.2] [6.7,3.3,5.7,2.1]
result = predictive_model( Features )
print(f"I predict {result} from Features {Features}")

I predict virginica (2) from Features [6.7, 3.3, 5.7, 2.1]


In [51]:
#
# try it on new data!  (grab it from the problem statement)
#
#
# TO DO for hw4pr1:
#       write a loop that will handle _multiple_ new flowers and predict their species...
#

LoF = [[5.7, 2.8, 4.1, 1.3],
[6.3, 3.3, 6.0, 2.5 ],
[6.2, 2.9, 4.3, 1.3 ],
[5.1, 2.5, 3.0, 1.1 ],
[5.4, 3.4, 1.5, 0.4 ],
[5.2, 4.1, 1.5, 0.1 ],
[5.8, 2.7, 5.1, 1.9 ],
[5.7, 2.9, 4.2, 1.3 ],
[4.8, 3.1, 1.6, 0.2 ]]

for x in LoF:
    result = predictive_model(x)
    print(f"I predict {result} from Features {x}")

I predict versicolor (1) from Features [5.7, 2.8, 4.1, 1.3]
I predict virginica (2) from Features [6.3, 3.3, 6.0, 2.5]
I predict versicolor (1) from Features [6.2, 2.9, 4.3, 1.3]
I predict versicolor (1) from Features [5.1, 2.5, 3.0, 1.1]
I predict setosa (0) from Features [5.4, 3.4, 1.5, 0.4]
I predict setosa (0) from Features [5.2, 4.1, 1.5, 0.1]
I predict virginica (2) from Features [5.8, 2.7, 5.1, 1.9]
I predict versicolor (1) from Features [5.7, 2.9, 4.2, 1.3]
I predict setosa (0) from Features [4.8, 3.1, 1.6, 0.2]


In [41]:
#
# Be sure your results from trying this on the unknown data are here - or above!
#

In [42]:
#
# That's it!  Welcome to the world of model-building workflows!!    
#
#             Our prediction?  We'll be back for more ML! 
#

#
# In fact, the rest of the hw is to run more ML workflows:   Digits, Titanic, Housing, ...
#